In [1]:
from typing import List
from math import pi
from pathlib import Path
import rospy
from copy import deepcopy
from datetime import datetime
import numpy as np

from moveit_commander import PlanningSceneInterface

from geometry_msgs.msg import (
    Pose,
    PoseStamped,
    Point,
    Quaternion,
)
from commander.msg import Goal
from commander.srv import (
    ExecuteTrajectory,
    PlanGoal,
    PlanGoalRequest,
    PlanSequence,
    PlanSequenceRequest,
    PickPlace,
    GetTcpPose,
    VisualizePoses,
    SetEe,
)

from commander.utils import poses_from_yaml, load_scene
from commander.transform_utils import (
    orient_poses,
    create_rotation_matrix,
    apply_transformation,
    create_translation_matrix,
)

from capture_manager.srv import CaptureToFile

CAPTURE = True

plan_goal_srv = rospy.ServiceProxy("commander/plan_goal", PlanGoal)
plan_sequence_srv = rospy.ServiceProxy("commander/plan_sequence", PlanSequence)
execute_trajectory_srv = rospy.ServiceProxy("commander/execute_trajectory", ExecuteTrajectory)
get_tcp_pose_srv = rospy.ServiceProxy("commander/get_tcp_pose", GetTcpPose)
set_ee_srv = rospy.ServiceProxy("commander/set_ee", SetEe)
pick_place_srv = rospy.ServiceProxy("commander/pick_place", PickPlace)

if CAPTURE:
    capture_to_file_srv = rospy.ServiceProxy("/capture_to_file", CaptureToFile)


def display_poses(poses: List[Pose], frame_id: str = "base_link") -> None:
    visualize_poses = rospy.ServiceProxy("/visualize_poses", VisualizePoses)
    visualize_poses(frame_id, poses)


rospy.init_node("robot_program")
scene = PlanningSceneInterface()
load_scene()

object


In [2]:
set_ee_srv("rgb_camera_tcp")

# cam_home = [0.0, -1.5708, 1.5708, -3.1416, -1.5708, 0]
cam_home = [0.0, -1.5708, 1.5708, 0, 1.5708, -pi]

vel = 0.2
acc = 0.2

sequence = poses_from_yaml("/dev_ws/src/commander/data/sequence_S.yaml")
origin = Pose(position=Point(0.75, 0.0, -0.7), orientation=Quaternion(0.0, 0.0, 0.0, 1.0))
sequence = orient_poses(origin, sequence)
tf = create_rotation_matrix([0, 0, pi])
sequence = [apply_transformation(pose, tf) for pose in sequence]
display_poses(sequence)

In [3]:
def generate_cap_sequence_S(origin: Pose, angle: float) -> List[Pose]:
    base = deepcopy(origin)
    base = apply_transformation(base, create_rotation_matrix([0, pi, -pi / 2]))

    back_rot = create_rotation_matrix([angle, 0.0, 0.0])
    front_rot = create_rotation_matrix([-angle, 0.0, 0.0])
    left_rot = create_rotation_matrix([0.0, -angle * 2, 0.0])
    right_rot = create_rotation_matrix([0.0, angle * 2, 0.0])

    focal_dist = create_translation_matrix([0.0, 0.0, -1.0])

    top = apply_transformation(deepcopy(base), focal_dist)
    back = apply_transformation(deepcopy(base), np.matmul(back_rot, focal_dist))
    left = apply_transformation(deepcopy(base), np.matmul(front_rot, focal_dist))
    front = apply_transformation(deepcopy(base), np.matmul(front_rot, focal_dist))
    side_left = apply_transformation(deepcopy(base), np.matmul(left_rot, focal_dist))
    side_right = apply_transformation(deepcopy(base), np.matmul(right_rot, focal_dist))

    return [
        top,
        back,
        # side_left,
        front,
        # side_right,
    ]


def generate_cap_sequence_L(origin: Pose, offset: float, angle: float) -> List[Pose]:
    base = deepcopy(origin)
    base = apply_transformation(base, create_rotation_matrix([0, pi, -pi / 2]))

    base_left = apply_transformation(deepcopy(base), create_translation_matrix([offset, 0.0, 0.0]))
    base_right = apply_transformation(
        deepcopy(base), create_translation_matrix([-offset, 0.0, 0.0])
    )

    back_rot = create_rotation_matrix([angle, 0.0, 0.0])
    front_rot = create_rotation_matrix([-angle, 0.0, 0.0])
    left_rot = create_rotation_matrix([0.0, -angle * 1.5, 0.0])
    right_rot = create_rotation_matrix([0.0, angle * 1.5, 0.0])

    focal_dist = create_translation_matrix([0.0, 0.0, -1.0])

    top_left = apply_transformation(deepcopy(base_left), focal_dist)
    top_right = apply_transformation(deepcopy(base_right), focal_dist)

    back_left = apply_transformation(deepcopy(base_left), np.matmul(back_rot, focal_dist))
    back_right = apply_transformation(deepcopy(base_right), np.matmul(back_rot, focal_dist))
    front_left = apply_transformation(deepcopy(base_left), np.matmul(front_rot, focal_dist))
    front_right = apply_transformation(deepcopy(base_right), np.matmul(front_rot, focal_dist))
    side_left = apply_transformation(deepcopy(base_left), np.matmul(left_rot, focal_dist))
    side_right = apply_transformation(deepcopy(base_right), np.matmul(right_rot, focal_dist))

    return [
        top_left,
        back_left,
        side_left,
        front_left,
        front_right,
        top_right,
        back_right,
        side_right,
    ]


cap_seq = generate_cap_sequence_S(origin, 0.35)
display_poses(cap_seq)

In [4]:
plan_goal_srv(Goal(joint_values=cam_home, vel_scale=vel, acc_scale=acc, planner="ptp"))

success: True
configuration_change: False

In [5]:
execute_trajectory_srv()

success: True

In [7]:
output_path = Path("/home/huanyu/capture")
if not output_path.exists():
    print("Output path does not exist")
dir_name = "log2"
dir_path = output_path / dir_name
dir_path.mkdir(parents=True, exist_ok=False)

In [84]:
sequence = cap_seq
for i in range(len(sequence)):
    # for i in range(1):
    resp = plan_goal_srv(Goal(pose=sequence[i], vel_scale=vel, acc_scale=acc, planner="ptp"))
    if resp.success:
        success = execute_trajectory_srv()
    else:
        rospy.logerr(f"Failed to plan pose{i}")
        break

    if CAPTURE:
        time = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
        tcp_pose = get_tcp_pose_srv().tcp_pose
        success = capture_to_file_srv(path=f"{dir_path}/{i}_{time}", capture_pose=tcp_pose)

environment


In [ ]:
set_ee_srv("rgb_camera_tcp")

cam_home = [0.0, -pi / 2.0, pi / 2.0, -pi / 2.0, -pi / 2.0, pi]

vel = 0.2
acc = 0.2

sequence = poses_from_yaml("/dev_ws/src/commander/data/sequence_env.yaml")
origin = Pose(position=Point(0.43, 0.0, 0.53), orientation=Quaternion(0.0, 0.0, 0.0, 1.0))
sequence = orient_poses(origin, sequence)
# tf = create_rotation_matrix([0, 0, pi])
# sequence = [apply_transformation(pose, tf) for pose in sequence]
display_poses(sequence)

In [ ]:
plan_goal_srv(Goal(joint_values=cam_home, vel_scale=0.1, acc_scale=0.1, planner="ptp"))

In [ ]:
execute_trajectory_srv()

In [ ]:
output_path = Path("/home/v/capture")
if not output_path.exists():
    print("Output path does not exist")
dir_name = "env"
dir_path = output_path / dir_name
dir_path.mkdir(parents=True, exist_ok=False)

In [ ]:
for i in range(len(sequence)):
    resp = plan_goal_srv(Goal(pose=sequence[i], vel_scale=0.1, acc_scale=0.1, planner="ptp"))
    if resp.success and not resp.configuration_change:
        success = execute_trajectory_srv()
    else:
        rospy.logerr(f"Failed to plan pose{i}")
        break

    if CAPTURE:
        time = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
        tcp_pose = get_tcp_pose_srv().tcp_pose
        success = capture_to_file_srv(path=f"{dir_path}/{i}_{time}", capture_pose=tcp_pose)